# **04 - modelo con preprocesado de otra forma y Random Forest**
## **Integrantes**
- SANTIAGO VILLEGAS NARANJO
- JENNY ANDREA OROZCO OSORIO
- XIOMARA GIRALDO PÉREZ

#**Librerias**

In [1]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 5.1 MB/s eta 0:00:00


In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
import time


In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"xiomaragiraldoprez","key":"27128718444a1ef0ccbc265ad597de45"}'}

In [4]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.39GB/s]


#**Descompresión e inspección del archivo**

In [5]:
!unzip udea*.zip > /dev/null

In [7]:
train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")

train.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


#**Limpieza y reemplazo de valores NAN para la columna E_HORASSEMANATRABAJA**

In [8]:
def organizar_horas_trabajadas_estudiante(data, columna):

    mapeo = {
        "0": 0,
        "menos de 10 horas": 1,
        "entre 11 y 20 horas": 2,
        "entre 21 y 30 horas": 3,
        "más de 30 horas": 4
    }

    valores_posibles = list(mapeo.keys())
    indices_nan = data[data[columna].isna()].index

    for i, idx in enumerate(indices_nan):
        data.at[idx, columna] = valores_posibles[i % len(valores_posibles)]

    data[columna] = data[columna].astype(str).str.strip().str.lower()

    data[columna] = data[columna].map(mapeo)

    return data

#**Limpieza y reemplzado de valores NAN para la columna E_HORASSEMANATRABAJA utilizando dummies**

In [9]:
def organizar_horas_trabajadas_estudiante_columna(data, columna):

    mapeo = {
        "0": 0,
        "menos de 10 horas": 1,
        "entre 11 y 20 horas": 2,
        "entre 21 y 30 horas": 3,
        "más de 30 horas": 4
    }

    valores_posibles = list(mapeo.keys())
    indices_nan = data[data[columna].isna()].index

    for i, idx in enumerate(indices_nan):
        data.at[idx, columna] = valores_posibles[i % len(valores_posibles)]

    dummies = pd.get_dummies(data[columna], prefix="Trabaja").astype(int)
    data = data.drop(columns=[columna]).join(dummies)

    return data

#**Limpieza y reemplazo de valores NAN para la columna E_PRGM_ACADEMICO**

In [10]:
def organizar_programa_estudiante(data, columna):
    categorias = {
        "salud": ["medicina", "enfermer", "fisioterap", "odontolog", "nutric", "terapia", "bacteriolog", "microbiolog", "farmac", "optometr", "salud"],
        "sociales": ["psicolog", "trabajo social", "sociolog","histor" "crimin", "politic", "juris", "derecho", "filosof","gobier","histori"],
        "administracion": ["admin", "gestión", "gestion", "logist", "mercadeo", "negocios", "marketing","direcc","gerenc"],
        "economia_finanzas": ["finanzas", "econom", "comercio", "merca"],
        "comunicacion": ["comunic", "periodismo", "medios", "publicidad","cine", "telev","fotogra","audio","publi"],
        "arte": ["arte", "musica", "teatro", "danza", "literatura", "bellas artes"],
        "educacion": ["licenc", "pedagog", "educac", "docencia","profes"],
        "diseño": ["dise","diseño","arquite"],
        "matematica": ["matema", "matemá", "estadis", "estadísti","ingenier"]
    }

    data[columna] = data[columna].astype(str).str.lower().str.strip()

    def clasificar(programa):
        for categoria, palabras_clave in categorias.items():
            if any(palabra in programa for palabra in palabras_clave):
                return categoria
        return "otras"

    data[columna] = data[columna].apply(clasificar)

    dummies = pd.get_dummies(data[columna], prefix="Programa").astype(int)
    data = data.drop(columns=[columna]).join(dummies)

    return data

#**Limpieza y reemplazado de valores NAN para la columna E_PRGM_DEPARTAMENTO**

In [11]:
def organizar_departamentos(data, columna):
  dummies = pd.get_dummies(data[columna], prefix="DEPARTAMENTO").astype(int)
  data = data.drop(columns=[columna]).join(dummies)

  return data

#**Limpieza y reemplazo de valores NAN para la columna F_ESTRATOVIVIENDA**

In [12]:
def organizar_estratos(data, columna):
  data[columna] = (
      data[columna]
      .str.lower()
      .str.replace("sin estrato", "0", regex=False)
      .str.replace("estrato ", "", regex=False))

  indices_nan = data[data[columna].isna()].index

  estratos = ["1", "2", "3", "4", "5", "6"]

  for i, idx in enumerate(indices_nan):
    data.at[idx, columna] = estratos[i % len(estratos)]

  data[columna] = (
    data[columna]
    .astype(int))

  return data

#**Limpieza y reemplazado de valores NAN para las columnas que tienen solo dos posibles valores**

In [13]:
def resumen_columna_binaria(data, columna, valores):
    si = (data[columna] == valores[0]).sum()
    no = (data[columna] == valores[1]).sum()
    nan = data[columna].isna().sum()
    total = len(data)

    si_pct = (si * 100) / total
    no_pct = (no * 100) / total
    nan_pct = (nan * 100) / total

    return (si_pct / 100), (nan_pct / 100)


def organizar_columna_binario_con_proporcion(data, columna, valores):
    n_missing = data[columna].isna().sum()

    if n_missing == 0:
        data[columna] = (data[columna]
                         .str.replace(valores[0], "1", regex=False)
                         .str.replace(valores[1], "0", regex=False)
                         .astype(int))
        print(f"No hay valores faltantes en la columna '{columna}'.")
        return data

    si_pct, nan_pct = resumen_columna_binaria(data, columna, valores)

    si = si_pct + (nan_pct / 2)
    no = 1 - si
    proporciones = [si, no]

    valores_aleatorios = np.random.choice(valores, size=n_missing, p=proporciones)
    data.loc[data[columna].isna(), columna] = valores_aleatorios

    data[columna] = (data[columna]
                     .str.replace(valores[0], "1", regex=False)
                     .str.replace(valores[1], "0", regex=False)
                     .astype(int))

    return data


#**Limpieza y reemplazo de valore NAN para la columna E_VALORMATRICULAUNIVERSIDAD utilizando dummies**

In [14]:
def organizar_valor_matricula_varias_colunas(data, columna):
  indices_nan = data[data[columna].isna()].index

  estratos = ["No pagó matrícula"]

  for i, idx in enumerate(indices_nan):
      data.at[idx, columna] = estratos[i % len(estratos)]

  dummies = pd.get_dummies(data[columna], prefix="Valor_").astype(int)
  data = data.drop(columns=[columna]).join(dummies)

  return data

#**Limpieza y reemplazo de valores NAN para la columna E_VALORMATRICULAUNIVERSIDAD**

In [15]:
def organizar_valor_matricula_en_una_columna(data, columna):
    mapeo = {
       'Entre 5.5 millones y menos de 7 millones':5,
       'Entre 2.5 millones y menos de 4 millones':3,
       'Entre 4 millones y menos de 5.5 millones':4,
       'Más de 7 millones':6,
       'Entre 1 millón y menos de 2.5 millones':2,
       'Entre 500 mil y menos de 1 millón':1,
       'Menos de 500 mil': 0,
       'No pagó matrícula':0
      }

    valores_posibles = list(mapeo.keys())
    indices_nan = data[data[columna].isna()].index

    for i, idx in enumerate(indices_nan):
        data.at[idx, columna] = valores_posibles[i % len(valores_posibles)]

    data[columna] = data[columna].astype(str).str.strip().str.lower()

    data[columna] = data[columna].map(mapeo)

    return data

#**Limpieza y reemplazo de valores NAN para la columna F_EDUCACIONPADRE**

In [16]:
def organizar_educacion_padre(data, columna):
  indices_nan = data[data[columna].isna()].index

  estratos = ['Técnica o tecnológica incompleta', 'Técnica o tecnológica completa',
  'Secundaria (Bachillerato) completa', 'No sabe', 'Primaria completa',
  'Educación profesional completa', 'Educación profesional incompleta',
  'Primaria incompleta', 'Postgrado',
  'Secundaria (Bachillerato) incompleta', 'Ninguno', 'No Aplica']

  for i, idx in enumerate(indices_nan):
      data.at[idx, columna] = estratos[i % len(estratos)]

  dummies = pd.get_dummies(data[columna], prefix="EducacionPadre").astype(int)
  data = data.drop(columns=[columna]).join(dummies)

  data=data.drop(columns=["EducacionPadre_No Aplica"])
  data=data.drop(columns=["EducacionPadre_No sabe"])
  data=data.drop(columns=["EducacionPadre_Ninguno"])
  data=data.drop(columns=["EducacionPadre_Técnica o tecnológica incompleta"])
  data=data.drop(columns=["EducacionPadre_Secundaria (Bachillerato) incompleta"])
  data=data.drop(columns=["EducacionPadre_Educación profesional incompleta"])
  data=data.drop(columns=["EducacionPadre_Primaria incompleta"])

  return data

#**Limpieza y reemplazo de valores NAN para la columna F_EDUCACIONMADRE**

In [17]:
def organizar_educacion_madre(data, columna):

  indices_nan = data[data[columna].isna()].index

  estratos = ['Postgrado', 'Técnica o tecnológica incompleta',
  'Secundaria (Bachillerato) completa', 'Primaria completa',
  'Técnica o tecnológica completa', 'Secundaria (Bachillerato) incompleta',
  'Educación profesional incompleta', 'Educación profesional completa',
  'Primaria incompleta','Ninguno', 'No Aplica', 'No sabe']

  for i, idx in enumerate(indices_nan):
      data.at[idx, columna] = estratos[i % len(estratos)]

  dummies = pd.get_dummies(data[columna], prefix="EducacionMadre").astype(int)
  data = data.drop(columns=[columna]).join(dummies)

  data=data.drop(columns=["EducacionMadre_No Aplica"])
  data=data.drop(columns=["EducacionMadre_No sabe"])
  data=data.drop(columns=["EducacionMadre_Ninguno"])
  data=data.drop(columns=["EducacionMadre_Técnica o tecnológica incompleta"])
  data=data.drop(columns=["EducacionMadre_Secundaria (Bachillerato) incompleta"])
  data=data.drop(columns=["EducacionMadre_Educación profesional incompleta"])
  data=data.drop(columns=["EducacionMadre_Primaria incompleta"])

  return data

#**Función donde se eliminan columnas innecesarias y se aplican las funciones de limpieza**

In [18]:
def clean_data(data):
  columnas_inecesarias = ["ID", "F_TIENELAVADORA", "INDICADOR_4", "PERIODO_ACADEMICO", "F_TIENEAUTOMOVIL", "F_TIENEINTERNET.1", "E_PRIVADO_LIBERTAD"]
  data=data.drop(columns=columnas_inecesarias)

  data=organizar_horas_trabajadas_estudiante(data, "E_HORASSEMANATRABAJA")
  data=organizar_programa_estudiante(data, "E_PRGM_ACADEMICO")
  data=organizar_departamentos(data, "E_PRGM_DEPARTAMENTO")
  data=organizar_estratos(data, "F_ESTRATOVIVIENDA")
  data=organizar_columna_binario_con_proporcion(data, "F_TIENEINTERNET", ["Si", "No"])
  data=organizar_columna_binario_con_proporcion(data, "F_TIENECOMPUTADOR", ["Si", "No"])
  data=organizar_columna_binario_con_proporcion(data, "E_PAGOMATRICULAPROPIO", ["Si", "No"])

  data=organizar_valor_matricula_varias_colunas(data, "E_VALORMATRICULAUNIVERSIDAD")
  data=organizar_educacion_padre(data, "F_EDUCACIONPADRE")
  data=organizar_educacion_madre(data, "F_EDUCACIONMADRE")
  return data

#Limpieza de los datos

In [19]:
#Guardar los IDs originales
test_ids = test["ID"].copy()

train_cleaned = clean_data(train)
test_cleaned = clean_data(test)

print(f"Dataset después de limpieza - Train: {train_cleaned.shape}, Test: {test_cleaned.shape}")


Aplicando limpieza inicial de datos...
Dataset después de limpieza - Train: (692500, 68), Test: (296786, 67)


# Definición de la variable objetivo

In [37]:
print(f"Train: {train.shape}")
print(f"Test: {test.shape}")

TARGET = "RENDIMIENTO_GLOBAL"
y = train[TARGET]
X = train.drop(columns=[TARGET])

# Análisis de la distribución de clases
print("\nDistribución de clases en y_train:")
print(y.value_counts(normalize=True))


Train: (692500, 21)
Test: (296786, 20)

Distribución de clases en y_train:
RENDIMIENTO_GLOBAL
alto          0.253601
bajo          0.249801
medio-bajo    0.248773
medio-alto    0.247825
Name: proportion, dtype: float64


# Preprocesado alternativo

In [38]:
def optimized_preprocess(df, is_train=True, label_encoders=None):
    df_processed = df.copy()

    # Selección de características más informativas
    categorical_cols = [
        'E_PRGM_ACADEMICO',
        'E_PRGM_DEPARTAMENTO',
        'F_ESTRATOVIVIENDA',
        'F_TIENEINTERNET',
        'F_TIENECOMPUTADOR',
        'E_VALORMATRICULAUNIVERSIDAD',
        'E_HORASSEMANATRABAJA',
        'F_EDUCACIONMADRE'
    ]

    if is_train:
        label_encoders = {}
        for col in categorical_cols:
            le = LabelEncoder()
            df_processed[col] = le.fit_transform(df_processed[col].astype(str))
            label_encoders[col] = le
    else:
        for col in categorical_cols:
            if col in label_encoders:
                df_processed[col] = df_processed[col].astype(str).map(
                    lambda x: label_encoders[col].transform([x])[0]
                    if x in label_encoders[col].classes_
                    else -1
                )

    # Características numéricas
    numeric_cols = ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4', 'PERIODO_ACADEMICO']

    # Feature engineering adicional
    if all(col in df_processed.columns for col in ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']):
        df_processed['indicador_promedio'] = df_processed[['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']].mean(axis=1)
        df_processed['indicador_max'] = df_processed[['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']].max(axis=1)
        df_processed['indicador_min'] = df_processed[['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']].min(axis=1)

    final_cols = categorical_cols + numeric_cols + ['indicador_promedio', 'indicador_max', 'indicador_min']
    df_processed = df_processed[final_cols]

    return df_processed, label_encoders


In [39]:
print("\nPreprocesando datos...")
X_processed, label_encoders = optimized_preprocess(X, is_train=True)


Preprocesando datos...


#División de datos

In [40]:
X_train, X_val, y_train, y_val = train_test_split(
    X_processed, y, test_size=0.2, stratify=y, random_state=42
)

print(f"\nDatos de entrenamiento: {X_train.shape}")
print(f"Datos de validación: {X_val.shape}")

print("\n" + "=" * 60)
print("ENTRENANDO MODELO GRADIENT BOOSTING (MEJOR ACCURACY)")
print("=" * 60)



Datos de entrenamiento: (554000, 16)
Datos de validación: (138500, 16)

ENTRENANDO MODELO GRADIENT BOOSTING (MEJOR ACCURACY)


#Modelo Random Forest  con parámetros optimizados

In [42]:
start_time = time.time()

gb_model = GradientBoostingClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=8,
    min_samples_split=200,
    min_samples_leaf=50,
    subsample=0.8,
    max_features='sqrt',
    random_state=42,
    verbose=1
)

print("Entrenando Gradient Boosting...")
gb_model.fit(X_train, y_train)
training_time = time.time() - start_time

print(f"Tiempo de entrenamiento: {training_time:.2f} segundos")

Entrenando Gradient Boosting...
      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.3677           0.0181           20.19m
         2           1.3503           0.0173           21.64m
         3           1.3369           0.0134           21.12m
         4           1.3243           0.0127           21.35m
         5           1.3143           0.0103           21.10m
         6           1.3049           0.0085           21.09m
         7           1.2975           0.0084           20.94m
         8           1.2902           0.0059           20.94m
         9           1.2847           0.0068           20.79m
        10           1.2797           0.0045           20.59m
        20           1.2485           0.0041           20.91m
        30           1.2312           0.0016           19.84m
        40           1.2205           0.0004           18.79m
        50           1.2125          -0.0016           18.37m
        60           1.2062          

#Validación


In [43]:
gb_predictions = gb_model.predict(X_val)
gb_accuracy = accuracy_score(y_val, gb_predictions)

print(f"\n{'='*60}")
print(f"GRADIENT BOOSTING ACCURACY: {gb_accuracy:.4f}")
print(f"{'='*60}")

print("\nReporte de clasificación detallado:")
print(classification_report(y_val, gb_predictions))

# Si Gradient Boosting no alcanza 0.45, probamos con Random Forest mejorado
if gb_accuracy < 0.45:
    print("\n" + "=" * 60)
    print("PROBANDO RANDOM FOREST MEJORADO")
    print("=" * 60)

    rf_improved = RandomForestClassifier(
        n_estimators=250,
        max_depth=20,
        min_samples_split=10,
        min_samples_leaf=4,
        max_features=0.7,
        bootstrap=True,
        class_weight='balanced',  # Importante para clases desbalanceadas
        random_state=42,
        n_jobs=-1,
        verbose=1
    )

    print("Entrenando Random Forest mejorado...")
    rf_improved.fit(X_train, y_train)

    rf_predictions = rf_improved.predict(X_val)
    rf_accuracy = accuracy_score(y_val, rf_predictions)

    print(f"\nRANDOM FOREST MEJORADO ACCURACY: {rf_accuracy:.4f}")
    print(classification_report(y_val, rf_predictions))

       # Seleccionar el mejor modelo
    if rf_accuracy > gb_accuracy:
        best_model = rf_improved
        best_accuracy = rf_accuracy
        print(f"✓ Seleccionado: Random Forest (Accuracy: {rf_accuracy:.4f})")
    else:
        best_model = gb_model
        best_accuracy = gb_accuracy
        print(f"✓ Seleccionado: Gradient Boosting (Accuracy: {gb_accuracy:.4f})")
else:
    best_model = gb_model
    best_accuracy = gb_accuracy
    print(f"✓ Gradient Boosting cumple el objetivo (Accuracy: {gb_accuracy:.4f})")

print("\n" + "=" * 60)
print("ENTRENANDO MODELO FINAL CON TODOS LOS DATOS")
print("=" * 60)



GRADIENT BOOSTING ACCURACY: 0.4259

Reporte de clasificación detallado:
              precision    recall  f1-score   support

        alto       0.54      0.62      0.58     35124
        bajo       0.46      0.54      0.49     34597
  medio-alto       0.32      0.26      0.29     34324
  medio-bajo       0.33      0.27      0.30     34455

    accuracy                           0.43    138500
   macro avg       0.41      0.42      0.41    138500
weighted avg       0.41      0.43      0.42    138500


PROBANDO RANDOM FOREST MEJORADO
Entrenando Random Forest mejorado...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 13.4min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 17.0min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    1.7s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    8.6s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:   10.6s finished



RANDOM FOREST MEJORADO ACCURACY: 0.4135
              precision    recall  f1-score   support

        alto       0.54      0.59      0.56     35124
        bajo       0.44      0.51      0.47     34597
  medio-alto       0.31      0.27      0.29     34324
  medio-bajo       0.32      0.28      0.30     34455

    accuracy                           0.41    138500
   macro avg       0.40      0.41      0.41    138500
weighted avg       0.40      0.41      0.41    138500

✓ Seleccionado: Gradient Boosting (Accuracy: 0.4259)

ENTRENANDO MODELO FINAL CON TODOS LOS DATOS


#Entrenando el mejor modelo

In [44]:
if best_model.__class__.__name__ == 'GradientBoostingClassifier':
    final_model = GradientBoostingClassifier(
        n_estimators=200,
        learning_rate=0.1,
        max_depth=8,
        min_samples_split=200,
        min_samples_leaf=50,
        subsample=0.8,
        max_features='sqrt',
        random_state=42
    )
else:
    final_model = RandomForestClassifier(
        n_estimators=250,
        max_depth=20,
        min_samples_split=10,
        min_samples_leaf=4,
        max_features=0.7,
        bootstrap=True,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    )

print(f"Entrenando modelo final ({final_model.__class__.__name__}) con todos los datos...")
final_model.fit(X_processed, y)

print("\n" + "=" * 60)
print("GENERANDO PREDICCIONES PARA KAGGLE")
print("=" * 60)

# Preprocesar datos de test
print("Preprocesando datos de test...")
test_processed, _ = optimized_preprocess(test, is_train=False, label_encoders=label_encoders)

# Asegurar que test tenga las mismas columnas
missing_cols = set(X_processed.columns) - set(test_processed.columns)
for col in missing_cols:
    test_processed[col] = 0

test_processed = test_processed[X_processed.columns]

print(f"Test procesado: {test_processed.shape}")

# Realizar predicciones
print("Realizando predicciones finales...")
final_predictions = final_model.predict(test_processed)


Entrenando modelo final (GradientBoostingClassifier) con todos los datos...

GENERANDO PREDICCIONES PARA KAGGLE
Preprocesando datos de test...
Test procesado: (296786, 16)
Realizando predicciones finales...


#Preparar entrega

In [45]:
# Crear archivo
submission = pd.DataFrame({
    "ID": test["ID"],
    "RENDIMIENTO_GLOBAL": final_predictions
})

# Guardar
submission_file = "submission_final.csv"
submission.to_csv(submission_file, index=False)

print(f"\nARCHIVO '{submission_file}' GENERADO CORRECTAMENTE!")
print(f"Accuracy esperado: {best_accuracy:.4f}")
print(f"Número de predicciones: {len(final_predictions)}")

# Información adicional
print(f"\n" + "=" * 60)
print("INFORMACIÓN DEL MODELO FINAL")
print("=" * 60)

print(f"Modelo utilizado: {final_model.__class__.__name__}")
print(f"Características utilizadas: {X_processed.shape[1]}")

if hasattr(final_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X_processed.columns,
        'importance': final_model.feature_importances_
    }).sort_values('importance', ascending=False)

    print("\nTop 10 características más importantes:")
    print(feature_importance.head(10))



ARCHIVO 'submission_final.csv' GENERADO CORRECTAMENTE!
Accuracy esperado: 0.4259
Número de predicciones: 296786

INFORMACIÓN DEL MODELO FINAL
Modelo utilizado: GradientBoostingClassifier
Características utilizadas: 16

Top 10 características más importantes:
                        feature  importance
5   E_VALORMATRICULAUNIVERSIDAD    0.226471
0              E_PRGM_ACADEMICO    0.209135
7              F_EDUCACIONMADRE    0.095958
1           E_PRGM_DEPARTAMENTO    0.089049
8                   INDICADOR_1    0.072572
14                indicador_max    0.047176
6          E_HORASSEMANATRABAJA    0.040194
9                   INDICADOR_2    0.039726
12            PERIODO_ACADEMICO    0.026817
11                  INDICADOR_4    0.025524


#Análisis

In [46]:
# Distribución de predicciones
print(f"\nDistribución de predicciones en test:")
print(pd.Series(final_predictions).value_counts(normalize=True))

print("\n" + "=" * 60)
print("¡PROCESO COMPLETADO!")
print("=" * 60)
print(" Sube el archivo 'submission_final.csv' a Kaggle")
print("Accuracy objetivo: > 0.45")
print(f"Accuracy esperado: {best_accuracy:.4f}")


Distribución de predicciones en test:
bajo          0.298939
alto          0.292510
medio-bajo    0.205579
medio-alto    0.202971
Name: proportion, dtype: float64

¡PROCESO COMPLETADO!
 Sube el archivo 'submission_final.csv' a Kaggle
Accuracy objetivo: > 0.45
Accuracy esperado: 0.4259


In [48]:
!kaggle competitions submit \
  -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia \
  -f submission_final.csv\
  -m "Envío final optimizado"


100% 4.06M/4.06M [00:00<00:00, 5.82MB/s]
Successfully submitted to UDEA/ai4eng 20252 - Pruebas Saber Pro Colombia